In [1]:
import sqlite3 as sql
import csv

In [26]:
def sql_connect(db_name=":memory:"):
    """
    Establishes a connection to an SQLite database.

    Args:
        db_name (str): The name of the database file. Defaults to ":memory:" for an in-memory database.
                      If a file-based database is desired, the file name should be provided (without the .db extension).

    Returns:
        tuple: 
            - (bool): True if connection is successful, False if it fails.
            - (sqlite3.Connection or None): The connection object if successful, or None if it fails.
            - (sqlite3.Cursor or None): The cursor object if successful, or None if it fails.
            - (str): The database name used in the connection (with .db appended for file-based DBs).

    Raises:
        sqlite3.Error: If the connection or cursor creation fails.
    """
    try:     
        # Establish connection
        connection = sql.connect(db_name)
        cursor = connection.cursor()
        
        # Return success
        return True, connection, cursor, db_name
    except sql.Error as e:
        # Print/log the error
        print(f"Database connection failed: {e}")
        return False, None, None, db_name


def create_table(conn, cursor, table_name, table_keys):
    """
    Creates a table in the SQLite database with an auto-incrementing primary key 'id'.

    Parameters:
    conn: sqlite3.Connection - The SQLite connection object
    cursor: sqlite3.Cursor - The SQLite cursor object
    table_name: str - The name of the table to be created
    table_keys: list - A list of tuples, where each tuple contains the column name and data type
    
    Example of table_keys: [("name", "TEXT"), ("age", "INTEGER")]
    """
    # Adding 'id' as the primary key and auto-incrementing column
    id_column = "id INTEGER UNIQUE PRIMARY KEY AUTOINCREMENT"
    
    # Constructing the SQL command for creating a table with additional user-provided keys
    keys_str = ", ".join([f"{col} {dtype}" for col, dtype in table_keys])
    
    # Final SQL statement includes the id column
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({id_column}, {keys_str});"
    
    try:
        # Executing the SQL command
        cursor.execute(create_table_query)
        conn.commit()
        print(f"Table '{table_name}' created successfully with auto-incrementing 'id'.")
    except sqlite3.Error as e:
        print(f"Error creating table: {e}")
    

def insert_csv_to_sqlite(conn, cursor, db_name, table_name, csv_file):
    """
    Inserts data from a CSV file into an existing SQLite3 table.

    :param db_name: Name of the SQLite3 database file.
    :param table_name: Name of the table in the SQLite3 database.
    :param csv_file: Path to the CSV file containing the data to be inserted.
    """
    try:
        # Open the CSV file
        with open(csv_file, mode = 'r') as file:
            reader = csv.reader(file)
            
            # Read the header (column names)
            headers = next(reader)
            
            # Create a placeholder string for the SQL INSERT statement (e.g., "?, ?, ?")
            placeholders = ', '.join(['?'] * len(headers))
            
            # Prepare the SQL insert statement
            insert_query = f"INSERT INTO {table_name} ({', '.join(headers)}) VALUES ({placeholders})"
            
            # Insert each row of data into the SQLite table
            for row in reader:
                cursor.execute(insert_query, row)
            
            # Commit the transaction
            conn.commit()
            print("Data successfully inserted into the table.")

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Close the connection
        if conn:
            conn.close()

In [28]:


db_name = 'materials.db'
table_name = 'joist_and_plank'
table_keys =[
    ('name', 'TEXT'),
    ('grade', 'TEXT'),
    ('fb', 'REAL'),
    ('fv', 'REAL'),
    ('fc', 'REAL'),
    ('fcp', 'REAL'),
    ('ft', 'REAL'),
    ('e', 'REAL'),
    ('e05', 'REAL'),
    ('tyoe', 'TEXT'),
]

csv_file = 'Joist_nand_Plank.csv'

connected, conn, cursor, db_name = sql_connect(db_name)
if connected:
    print(f"Successfully connected to {db_name}")
    create_table(conn, cursor, table_name, table_keys)
    insert_csv_to_sqlite(conn, cursor, db_name, table_name, csv_file)

Successfully connected to materials.db
Table 'joist_and_plank' created successfully with auto-incrementing 'id'.


NameError: name 'sqlite3' is not defined